In [ ]:
import numpy as np
import pandas as pd
import io
import plotly.express as px
import plotly.offline as pyo
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff
# %pip install jieba
#%pip install keras==2.6.0
#%pip install openpyxl
%pip install jieba
import keras
import jieba




In [ ]:
pd.options.display.max_rows=500

In [ ]:
jieba.set_dictionary('dict.txt.big.txt')    #詞庫
# jieba.load_userdict('user_dict.txt') 

In [ ]:
def preprocess(question):
    """
    :param question:
    :type question:
    :return:
    :rtype:
    """
    #     words = list(jieba.cut(question)) ## 精準模式
    words = jieba.lcut(question, cut_all=False)  ## 全斷詞模式
    return words

In [ ]:
df = pd.read_excel('NLP.xlsx')
df['發文標題_斷']=df['發文標題'].astype(str).apply(lambda x:preprocess(x))
def delete_unrelated(strs):
    dels=["【","】","（","）","《","》","!","?","？","、","(",")","!","！","|","，",'-','_','[',']']
    for i in range(len(dels)):
        strs=strs.replace(dels[i], "")
    return strs


df['發文標題_斷']=(df['發文標題'].astype(str)
                                .apply(lambda x:delete_unrelated(x))
                                .apply(lambda x:preprocess(x)))
                                
df=df.rename(columns={'發文標題':'Title',"發文標題_斷":'Title_f'})
df.head(300)

若要再進一步斷詞，可以將台灣所有大學的名稱，以及一般簡稱加入dict中加以斷詞。

# 標題分類

對於文本分類問題的分類訪法，想要獲取大量的標註數據，一般有幾種方法

1.人工標籤，費時費力也需大量成本。

2.取得其餘類似平台的數據分類標籤，[一種為，標題和標籤一起拿，另一種則是指拿標籤，自己去對要訓練的題目做分類]

3.自己寫function嘗試分類。 (不夠詳盡的話準確率會很低)

觀察提問標題之後，認為提問大概能分幾類：

1.感情
2.課業
3.美食
4.社團
5.生活
6.其他

自己嘗試寫函式做分類，對於大眾發問習慣要有一定認知，才能將優先級決定好。

In [ ]:
def categorys(li:list):
    club=['友會','社團','迎新','宿營']

    lesson=['選課','英文','化學','教學','通識',"體育",'必修','主修','輔系',
            '學分','畢業','選修','研究所','推甄','出席率',]

    love=['感情','約會']

    life=['宿舍','男宿','女宿','繳費','繳款','通勤','校園','方便',
    '混宿','打工','在外','賺錢','附近','投資',
    '交友',"學費","金融","疫情",'生活用品','未來','優惠','推薦','機車']

    food=['早餐','晚餐','晚餐','餐廳','自助餐','素食','臭豆腐','美食']
    club_count=0
    lesson_count=0
    love_count=0
    life_count=0
    food_count=0
    for i in li:
        if i in life:
            # label='生活'
            life_count+=1
        elif i in lesson:
            # label="課業"
            lesson_count+=1
        elif i in food:
            # label="美食"
            food_count+=1
        elif i in club:
            # label='社團'
            club_count+=1
        elif i in love:
            # label='感情'
            love_count+=1
        else:
            pass
    c_li=[club_count,lesson_count,love_count,life_count,food_count,]
    if life_count==0 and lesson_count==0 and love_count==0 and food_count==0 and club_count==0:
        label='其他'
    else:
        maxvalue=max(c_li)
        if c_li.index(maxvalue)==0:
            label='社團'
        elif c_li.index(maxvalue)==1:
            label="課業"
        elif c_li.index(maxvalue)==2:
            label='感情'
        elif c_li.index(maxvalue)==3:
            label='生活'    
        elif c_li.index(maxvalue)==4:
            label="美食"  
    
    return label
            

In [ ]:
df['label']=df['Title_f'].apply(lambda x:categorys(x))
df[['Title','label']].head(200)

## 後續還能如何做

1.將斷詞後的詞彙全部倒在在一起
2.建立一個空字典
3.查看所有新聞標題，裡面每出現一個字典裏面沒有的詞彙，就為該才會指定一個字典裡面還沒有的索引數字，並將該詞會放入字典。
4.利用建好的dict，將每個新聞標題裏頭包含的詞彙轉成數字。

In [ ]:
MAX_NUM_WORDS=10000

tokenizer=keras.preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS)

In [ ]:
dict1=df.Title_f
dict1.shape

In [ ]:
tokenizer.fit_on_texts(dict1)

In [ ]:
x_train=tokenizer.texts_to_sequences(dict1)
len(x_train)

In [ ]:
for seq in x_train[:1]:
  print([tokenizer.index_word[idx] for idx in seq])

# Zero Padding

雖然我們已經將每個新聞標題的文本轉為一行行的數字序列， 但每篇標題的序列長度並不相同

In [ ]:
for seq in x_train[:10]:
    print(len(seq), seq[:5], ' ...')

In [ ]:
max_seq_len = max([
    len(seq) for seq in x_train])
max_seq_len

若之後要進行NLP的模型處理，我們一班要設定一個MAX_SEQUENCE_LENGTH來讓所有序列長度一致

長度超過此數字的序列尾巴會被刪掉；而針對原來長度不足的序列，我們則會在詞彙前面補零。Keras有個方便函式 pad_sequences 來幫助我們完成這件工作：

In [ ]:
MAX_SEQUENCE_LENGTH=53 #一般來說，是設定為最長的長度，避免資料缺失

x_train=keras.preprocessing.sequence.pad_sequences(x_train,maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
x_train[:5]

# 標題分類

對於文本分類問題的分類訪法，想要獲取大量的標註數據，一般有幾種方法

1.人工標籤，費時費力也需大量成本。

2.取得其餘類似平台的數據分類標籤，[一種為，標題和標籤一起拿，另一種則是指拿標籤，自己去對要訓練的題目做分類]

3.自己寫function嘗試分類。 (不夠詳盡的話準確率會很低)

觀察提問標題之後，認為提問大概能分幾類：

1.感情
2.課業
3.美食
4.社團
5.生活
6.其他

自己嘗試寫函式做分類，對於大眾發問習慣要有一定認知，才能將優先級決定好。

In [125]:
def categorys(li:list):
    club=['友會','社團','迎新','宿營']

    lesson=['選課','英文','化學','教學','通識',"體育",'必修','主修','輔系',
            '學分','畢業','選修','研究所','推甄','出席率',]

    love=['感情','約會']

    life=['宿舍','男宿','女宿','繳費','繳款','通勤','校園','方便',
    '混宿','打工','在外','賺錢','附近','投資',
    '交友',"學費","金融","疫情",'生活用品','未來','優惠','推薦','機車']

    food=['早餐','晚餐','晚餐','餐廳','自助餐','素食','臭豆腐','美食']
    club_count=0
    lesson_count=0
    love_count=0
    life_count=0
    food_count=0
    for i in li:
        if i in life:
            # label='生活'
            life_count+=1
        elif i in lesson:
            # label="課業"
            lesson_count+=1
        elif i in food:
            # label="美食"
            food_count+=1
        elif i in club:
            # label='社團'
            club_count+=1
        elif i in love:
            # label='感情'
            love_count+=1
        else:
            pass
    c_li=[club_count,lesson_count,love_count,life_count,food_count,]
    if life_count==0 and lesson_count==0 and love_count==0 and food_count==0 and club_count==0:
        label='其他'
    else:
        maxvalue=max(c_li)
        if c_li.index(maxvalue)==0:
            label='社團'
        elif c_li.index(maxvalue)==1:
            label="課業"
        elif c_li.index(maxvalue)==2:
            label='感情'
        elif c_li.index(maxvalue)==3:
            label='生活'    
        elif c_li.index(maxvalue)==4:
            label="美食"  
    
    return label
            

In [124]:
df['label']=df['Title_f'].apply(lambda x:categorys(x))
df[['Title','label']].head(200)

,Title,label
0,【逢甲大學】推薦的機車行有哪些？（上大學機車托運）,生活
1,【崑山科大】上大學穿短裙有腳毛跟腋毛好尷尬，有沒有推薦什麼除毛的方式?,生活
2,什麼是輔系、雙主修？,課業
3,要參加系隊嗎,其他
4,【中正大學】雖然地處偏僻 卻好像該有的都有？,其他
5,【亞洲大學】服務學習因意外無法做怎麼辦？,其他
6,大學生投資股票？,生活
7,【朝陽科技大學】的圖書館設施有哪些呢？,其他
8,外幣存錢適合嗎？,其他
9,???出國留學的計畫嗎？,其他


## 後續還能如何做

1.將斷詞後的詞彙全部倒在在一起
2.建立一個空字典
3.查看所有新聞標題，裡面每出現一個字典裏面沒有的詞彙，就為該才會指定一個字典裡面還沒有的索引數字，並將該詞會放入字典。
4.利用建好的dict，將每個新聞標題裏頭包含的詞彙轉成數字。

In [46]:
MAX_NUM_WORDS=10000

tokenizer=keras.preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS)

In [47]:
dict1=df.Title_f
dict1.shape

(9561,)

In [48]:
tokenizer.fit_on_texts(dict1)

In [49]:
x_train=tokenizer.texts_to_sequences(dict1)
len(x_train)

9561

In [50]:
for seq in x_train[:1]:
  print([tokenizer.index_word[idx] for idx in seq])

['逢甲', '大學', '推薦', '的', '機車行', '有', '哪些', '？', '上大學', '機車', '托運']


# Zero Padding

雖然我們已經將每個新聞標題的文本轉為一行行的數字序列， 但每篇標題的序列長度並不相同

In [51]:
for seq in x_train[:10]:
    print(len(seq), seq[:5], ' ...')

11 [40, 5, 13, 3, 1505]  ...
23 [4302, 1297, 21, 74, 131]  ...
7 [10, 25, 292, 69, 367]  ...
4 [18, 135, 511, 2]  ...
14 [35, 5, 2120, 4304, 1755]  ...
10 [216, 5, 383, 2803, 1756]  ...
4 [14, 407, 939, 1]  ...
10 [151, 123, 5, 3, 159]  ...
5 [4305, 1157, 169, 2, 1]  ...
9 [4, 4, 4, 532, 940]  ...


In [52]:
max_seq_len = max([
    len(seq) for seq in x_train])
max_seq_len

53

若之後要進行NLP的模型處理，我們一班要設定一個MAX_SEQUENCE_LENGTH來讓所有序列長度一致

長度超過此數字的序列尾巴會被刪掉；而針對原來長度不足的序列，我們則會在詞彙前面補零。Keras有個方便函式 pad_sequences 來幫助我們完成這件工作：

In [18]:
MAX_SEQUENCE_LENGTH=53 #一般來說，是設定為最長的長度，避免資料缺失

x_train=keras.preprocessing.sequence.pad_sequences(x_train,maxlen=MAX_SEQUENCE_LENGTH)

In [19]:
x_train[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    8,   43,    5,    7,   15,    3,
        1502,    9,   56,    1,   86,   78,  162, 4273,   81],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    8, 4274, 1296,   23,    7,
          78,  137, 4275, 2789, 1297,   51, 2790,   46, 1020,   21,    9,
          89,    9,   15,   12, 1751, 1297,    3,  132,    4],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0